In [1]:
### Mount Notebook to Google Drive
from google.colab import drive
drive.mount('/content/drive')
# change the working directory to the Drive root
%cd /content/drive/My\ Drive/Colab\ Notebooks/deep-agents-from-scratch-main

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/deep-agents-from-scratch-main


In [2]:
!pip install --quiet -U langchain-google-genai langchain_core langgraph langchain langsmith

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage

In [3]:
import os

from dotenv import load_dotenv

load_dotenv(os.path.join("..", ".env"), override=True)

# %load_ext autoreload
# %autoreload 2

True

## Context Isolation: Sub-agents

<img src="./assets/agent_header_subagent.png" width="800" style="display:block; margin-left:0;">

Agent context can grow quickly as conversations progress, leading to several long context-related problems. A primary issue is context clash or confusion, where mixed objectives within the same context window can lead to suboptimal performance. [Context isolation](https://blog.langchain.com/context-engineering-for-agents/) provides an effective solution by delegating tasks to [specialized sub-agents](https://www.anthropic.com/engineering/multi-agent-research-system), each operating within their own isolated context window. This approach prevents context clashes, confusion, poisoning, and dilution while enabling focused, specialized task execution.



### Sub-agent delegation
![./assets/subagents.png](./assets/subagents.png)
The primary insight is that we can create sub-agents with different tool sets tailored to specific tasks. Each sub-agent is stored in a registry dictionary with `subagent_type` as the key, allowing the main agent to delegate work through a `task(description, subagent_type)` tool call. The sub-agent operates in complete isolation from the parent's context, and its results are returned as a `ToolMessage` to the parent agent, maintaining clean separation of concerns.

## Step 1: Create Sub Agents

Let's define how the user will specify sub agents
```python
from typing_extensions import TypedDict

class SubAgent(TypedDict):
    """Configuration for a specialized sub-agent."""

    name: str
    description: str
    prompt: str
    tools: NotRequired[list[str]]
```

We will use a list of these objects to create all the sub agents we have access to

```python
agents: list[SubAgent] = ...
subagents = {
    agent['name']: create_react_agent(
        model=model,
        prompt=agent['prompt'],
        tools = get_tools(agent['tools']),
        ...
    )
}
```

## Step 2: Create a tool to use Sub Agents

Logically, should look something like:

```python
def task(
    description: str  # The task the subagent should do
    subagent_type: str  # Which subagent to use
):
    # Create new messages to pass to subagent - should just be the description
    # Call sub agent
    # Update state with both the subagents response AND any changes to file system
```

This ends up in full looking like:

```python
@tool(description=TASK_DESCRIPTION_PREFIX.format(other_agents=other_agents_string))
def task(
    description: str,
    subagent_type: str,
    state: Annotated[DeepAgentState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId],
):
    """Delegate a task to a specialized sub-agent with isolated context.

    This creates a fresh context for the sub-agent containing only the task description,
    preventing context pollution from the parent agent's conversation history.
    """
    # Validate requested agent type exists
    if subagent_type not in agents:
        return f"Error: invoked agent of type {subagent_type}, the only allowed types are {[f'`{k}`' for k in agents]}"

    # Get the requested sub-agent
    sub_agent = agents[subagent_type]

    # Create isolated context with only the task description
    # This is the key to context isolation - no parent history
    state["messages"] = [{"role": "user", "content": description}]

    # Execute the sub-agent in isolation
    result = sub_agent.invoke(state)

    # Return results to parent agent via Command state update
    return Command(
        update={
            "files": result.get("files", {}),  # Merge any file changes
            "messages": [
                # Sub-agent result becomes a ToolMessage in parent context
                ToolMessage(
                    result["messages"][-1].content, tool_call_id=tool_call_id
                )
            ],
        }
    )

return task
```

In [4]:
%%writefile ./src/deep_agents_from_scratch/task_tool.py
"""Task delegation tools for context isolation through sub-agents.

This module provides the core infrastructure for creating and managing sub-agents
with isolated contexts. Sub-agents prevent context clash by operating with clean
context windows containing only their specific task description.
"""

from typing import Annotated, NotRequired
from typing_extensions import TypedDict

from langchain_core.messages import ToolMessage
from langchain_core.tools import BaseTool, InjectedToolCallId, tool
from langgraph.prebuilt import InjectedState, create_react_agent
from langgraph.types import Command

from src.deep_agents_from_scratch.prompts import TASK_DESCRIPTION_PREFIX
from src.deep_agents_from_scratch.state import DeepAgentState


class SubAgent(TypedDict):
    """Configuration for a specialized sub-agent."""

    name: str
    description: str
    prompt: str
    tools: NotRequired[list[str]]


def _create_task_tool(tools, subagents: list[SubAgent], model, state_schema):
    """Create a task delegation tool that enables context isolation through sub-agents.

    This function implements the core pattern for spawning specialized sub-agents with
    isolated contexts, preventing context clash and confusion in complex multi-step tasks.

    Args:
        tools: List of available tools that can be assigned to sub-agents
        subagents: List of specialized sub-agent configurations
        model: The language model to use for all agents
        state_schema: The state schema (typically DeepAgentState)

    Returns:
        A 'task' tool that can delegate work to specialized sub-agents
    """
    # Create agent registry
    agents = {}

    # Build tool name mapping for selective tool assignment
    tools_by_name = {}
    for tool_ in tools:
        if not isinstance(tool_, BaseTool):
            tool_ = tool(tool_)
        tools_by_name[tool_.name] = tool_

    # Create specialized sub-agents based on configurations
    for _agent in subagents:
        if "tools" in _agent:
            # Use specific tools if specified
            _tools = [tools_by_name[t] for t in _agent["tools"]]
        else:
            # Default to all tools
            _tools = tools
        agents[_agent["name"]] = create_react_agent(
            model, prompt=_agent["prompt"], tools=_tools, state_schema=state_schema
        )

    # Generate description of available sub-agents for the tool description
    other_agents_string = [
        f"- {_agent['name']}: {_agent['description']}" for _agent in subagents
    ]

    @tool(description=TASK_DESCRIPTION_PREFIX.format(other_agents=other_agents_string))
    def task(
        description: str,
        subagent_type: str,
        state: Annotated[DeepAgentState, InjectedState],
        tool_call_id: Annotated[str, InjectedToolCallId],
    ):
        """Delegate a task to a specialized sub-agent with isolated context.

        This creates a fresh context for the sub-agent containing only the task description,
        preventing context pollution from the parent agent's conversation history.
        """
        # Validate requested agent type exists
        if subagent_type not in agents:
            return f"Error: invoked agent of type {subagent_type}, the only allowed types are {[f'`{k}`' for k in agents]}"

        # Get the requested sub-agent
        sub_agent = agents[subagent_type]

        # Create isolated context with only the task description
        # This is the key to context isolation - no parent history
        state["messages"] = [{"role": "user", "content": description}]

        # Execute the sub-agent in isolation
        result = sub_agent.invoke(state)

        # Return results to parent agent via Command state update
        return Command(
            update={
                "files": result.get("files", {}),  # Merge any file changes
                "messages": [
                    # Sub-agent result becomes a ToolMessage in parent context
                    ToolMessage(
                        result["messages"][-1].content, tool_call_id=tool_call_id
                    )
                ],
            }
        )

    return task

Overwriting ./src/deep_agents_from_scratch/task_tool.py


Now, you have a routine that will generate sub-agents as tools. Now, you can define specific sub-agents and allow the system to call them with the `task` tool.    
Above, the `_create_task_tool` receives a list of type `SubAgent`. This list contains descriptions of the agents that are to be created.

```python
class SubAgent(TypedDict):
    """Configuration for a specialized sub-agent."""

    name: str
    description: str
    prompt: str
    tools: NotRequired[list[str]]


def _create_task_tool(tools, subagents: list[SubAgent], model, state_schema):
    """Create a task delegation tool that enables context isolation through sub-agents.

```
The `SubAgent` class defines the unique information needed to satisfy the dual role of a sub-agent. Sub-agents act as both tools and agents.  

- **As tools**, they provide the supervisor agent with information about their capabilities and how they can be called.  
- **As agents**, they require a prompt that describes how to carry out their tasks, along with a set of tools targeted for those tasks.  

Below, you will create a research subagent. Its `description` informs the supervisor agent that a single task should be delegated to this sub-agent. The `SIMPLE_RESEARCH_INSTRUCTIONS` is a prompt that is used by the sub-agent to direct its research. In this example, it is brief, but for a general-purpose researcher, it could be much more detailed. The sub-agent is also supplied with a `web_search` tool to use during its research.  

```python
# Create research sub-agent
research_sub_agent = {
    "name": "research-agent",
    "description": "Delegate research to the sub-agent researcher. Only give this researcher one topic at a time.",
    "prompt": SIMPLE_RESEARCH_INSTRUCTIONS,
    "tools": ["web_search"],
}
```

Note that the sub-agent receives a specific task, along with the necessary tools to complete it. It operates in its own context, limited to the single task description. This [context-engineering](https://blog.langchain.com/context-engineering-for-agents/) approach ensures that the subagent’s working context remains free of context clashes, confusion, poisoning, and dilution.

The supervisor agent prompt must now include a descripition of how to invoke and use these sub-agents. This is shown below. Note the *Available Tools* description and the instructions to use parallel research where applicable.

In [5]:
from notebooks.utils import show_prompt

from src.deep_agents_from_scratch.prompts import SUBAGENT_USAGE_INSTRUCTIONS

show_prompt(SUBAGENT_USAGE_INSTRUCTIONS)

╭─────────────────────────────────── Prompt ───────────────────────────────────╮
│                                                                              │
│  You can delegate tasks to sub-agents.                                       │
│                                                                              │
│  <Task>                                                                      │
│  Your role is to coordinate research by delegating specific research tasks   │
│  to sub-agents.                                                              │
│  </Task>                                                                     │
│                                                                              │
│  <Available Tools>                                                           │
│  1. **task(description, subagent_type)**: Delegate research tasks to         │
│  specialized sub-agents                                                      │
│     - description: Clear, 

Let's now build a research system with a supervisor and sub-agents. This will just be a mock-up version with pre-defined search results to demonstrate how the pieces go together. In the next lesson, you will build a full-fledged research system.

In [9]:
from utils import format_messages

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Give me an overview of Model Context Protocol (MCP).",
            }
        ],
    }
)

format_messages(result["messages"])

╭────────────────────────────────── 🧑 Human ──────────────────────────────────╮
│ Give me an overview of Model Context Protocol (MCP).                         │
╰──────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────── 📝 AI ────────────────────────────────────╮
│                                                                              │
│                                                                              │
│ 🔧 Tool Call: task                                                           │
│    Args: {                                                                   │
│   "subagent_type": "research-agent",                                         │
│   "description": "Provide an overview of Model Context Protocol (MCP)."      │
│ }                                                                            │
│    ID: c3c65832-e347-43e3-b4b0-bb32c856151c                                  │
╰──────────────────────────────

Trace:
https://smith.langchain.com/public/26cc1c2b-e785-4c6d-a2a7-c30a31875fc7/r
<!-- https://smith.langchain.com/public/edc4e672-db9c-457a-953d-f62e7813591c/r -->